In [5]:
import tensorboard.backend.event_processing.event_accumulator as ec

files = [
"/home/sontung/thesis/location-based-generative/logs/20200725-100335/events.out.tfevents.1595664215.ipvs8gtx.24823.0",
"/home/sontung/thesis/location-based-generative/logs/sim20200725-114336/events.out.tfevents.1595670216.ipvs8gtx.32638.0",
]

eas = []
for f in files:
    ea = ec.EventAccumulator(f)
    ea.Reload()
    eas.append(ea)

print(eas[0].Tags())

{'images': [], 'audio': [], 'histograms': [], 'scalars': ['train/loss', 'train/iou', 'val/loss', 'val/acc', 'val/iou'], 'distributions': [], 'tensors': [], 'graph': False, 'meta_graph': False, 'run_metadata': []}


In [24]:
data2 = [[du.value for du in ea.Scalars("train/iou")] for ea in eas]
data1 = [[du.value for du in ea.Scalars("train/loss")] for ea in eas]
print(data1)
print([i[-1] for i in data1])

[[0.0688549131155014, 0.04856899008154869, 0.057940565049648285, 0.04846090078353882, 0.049644120037555695, 0.056259553879499435, 0.044313155114650726, 0.03873211517930031, 0.0387047603726387, 0.032347679138183594, 0.04013942927122116, 0.06606067717075348, 0.05849655345082283, 0.055466488003730774, 0.05285181850194931, 0.04384123906493187, 0.04579120874404907, 0.03932268172502518, 0.03475028648972511, 0.03139714524149895, 0.03689220920205116, 0.02996782585978508, 0.02705065906047821, 0.032034777104854584, 0.026189079508185387, 0.023172466084361076, 0.021404847502708435, 0.016986533999443054, 0.015635572373867035, 0.017309479415416718, 0.021147295832633972, 0.013004055246710777, 0.01516338624060154, 0.014340108260512352, 0.014957073144614697, 0.013252084143459797, 0.008953984826803207, 0.010991329327225685, 0.009812946431338787, 0.01042944472283125, 0.008315825834870338, 0.013601226732134819, 0.010965218767523766, 0.012933716177940369, 0.01317065954208374, 0.012108867056667805, 0.010186

In [29]:
import numpy as np
import json
import sys
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly

plotly.io.orca.config.executable = '/home/sontung/tools/orca-1.3.1.AppImage'


fig = make_subplots(rows=1, cols=2, subplot_titles=("on PBW data", "on sim data"), shared_yaxes=True)

names = ["IoU", "MSE loss", "only kld"]
colors = [dict(color='firebrick'), dict(color='royalblue'), dict(color='green')]

fig.add_trace(
    go.Scatter(
        x=list(range(len(data_entry))),
        y=data2[0],
        mode="lines",
        name=names[0],
        line=colors[2],
        showlegend=True
    ), row=1, col=1,
)

fig.add_trace(
    go.Scatter(
        x=list(range(len(data_entry))),
        y=data2[1],
        mode="lines",
        name=names[0],
        line=colors[2],
        showlegend=False
    ), row=1, col=2
)

# fig.add_trace(
#     go.Scatter(
#         x=list(range(len(data_entry))),
#         y=data1[0],
#         mode="lines",
#         name=names[0],
#         line=colors[0],
#         showlegend=True
#     ), row=1, col=1,
# )

# fig.add_trace(
#     go.Scatter(
#         x=list(range(len(data_entry))),
#         y=data1[1],
#         mode="lines",
#         name=names[0],
#         line=colors[0],
#         showlegend=False
#     ), row=1, col=2
# )

fig.update_xaxes(title_text="iteration", row=1, col=1)
fig.update_xaxes(title_text="iteration", row=1, col=2)

fig.update_xaxes(ticks="outside")
fig.update_yaxes(ticks="outside", col=1)

fig.update_layout(
    yaxis_title="IoU",
    legend=dict(x=0.0, y=-0.25),

    font=dict(
        family="Courier New, monospace",
        size=18,
        color="#000000"
    ),
    paper_bgcolor='rgba(0,0,0,0)',
    margin=dict(l=80, r=10, t=30, b=80)

#     plot_bgcolor='rgba(0,0,0,0)'
)
fig.update_layout(legend_orientation="h")

fig.show()
fig.write_image("learning.png", width=1000, height=None, scale=2)
